<a href="https://colab.research.google.com/github/YuehMintTai/GAETest20210722/blob/main/GAETry20210723.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/YuehMintTai/GAETest20210722.git

Cloning into 'GAETest20210722'...
remote: Enumerating objects: 103, done.
remote: Counting objects: 100% (103/103), done.
remote: Compressing objects: 100% (98/98), done.
remote: Total 103 (delta 29), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (103/103), 47.59 KiB | 4.76 MiB/s, done.
Resolving deltas: 100% (29/29), done.


In [3]:
import pandas as pd
import numpy as np
import flask 
print('pandas==',pd.__version__)
print('numpy==',np.__version__)
print('flask==',flask.__version__)

pandas== 1.1.5
numpy== 1.19.5
flask== 1.1.4
